In [ ]:
# commands used to pull down CSV:
# gtt report --report=records --output=csv --no_headlines -c --time_format="[%sign][%hours_overall]" msoe.edu/sdl/create-institute/backend --file="./gitlab-time-tracker-backend.csv"
# gtt report --report=records --output=csv --no_headlines -c --time_format="[%sign][%hours_overall]" msoe.edu/sdl/create-institute/frontend --file="./gitlab-time-tracker-frontend.csv"

import pandas as pd
import numpy as np
from math import isnan

# fix notebook styling
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

df_frontend = pd.read_csv('./gitlab-time-tracker-frontend.records.csv')
df_backend = pd.read_csv('./gitlab-time-tracker-backend.records.csv')
df_frontend['repo'] = 'frontend'
df_backend['repo'] = 'backend'
df = pd.concat([df_frontend, df_backend], sort=True)

def create_link(x):
    if x.type == "Issue":
        return f"https://gitlab.com/msoe.edu/sdl/create-institute/{x.repo}/issues/{x.iid}"
    else:
        return ""

format_hours = lambda x: '0' if x == 0 else "{0:.2f}".format(x).rstrip('0').rstrip('.') + 'h'

df['link'] = df.apply(create_link, axis=1)
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df.sort_values(by=['date', 'user'], inplace=True, ascending=False)

df = df.reset_index(drop=True)

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.set_option('max_colwidth', 800)

df.style.format({
    'time': format_hours
})

In [ ]:
df['year'] = df['date'].dt.strftime('%Y')
df['month'] = df['date'].dt.strftime('%m')
df['day'] = df['date'].dt.strftime('%d (%a)')
df['week'] = df['date'].dt.strftime('%W')

res = df.pivot_table(
    index='user',
    columns=['year', 'week', 'month', 'day'],
    values='time',
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
    margins_name='Σ'
)
res.style.format(format_hours)

In [ ]:
res_weeks = df.pivot_table(
    index='user',
    columns=['week'],
    values='time',
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
    margins_name='Σ'
)
res_weeks.style.format(format_hours)

In [ ]:
# show number of days in each numbered week (sanity check)
res_weeks = df.groupby(['week'])['day'].nunique()
res_weeks